In [0]:
import $file.common
import spark._
import common._
import org.apache.spark.sql.functions._
import org.apache.spark.rdd._
import org.apache.spark.sql.types.{IntegerType, StringType, StructType}
import org.apache.spark.sql.functions.{col, to_date}
import spark.implicits._
import spark.sqlContext.implicits._

Cannot resolve $file import: /home/jovyan/work/common.sc

: 

In [ ]:
import plotly._
import plotly.element._
import plotly.layout._
import plotly.Almond._

# Media de infecciones por Km2 

### Utilizando RDDs

In [ ]:
def infections(lines : RDD[String]) : RDD[Infection] =
    lines.map(line => {
      val arr = line.split(",")
      Infection(
        day = arr(1).toInt,
        month = arr(2).toInt,
        year = arr(3).toInt,
        nCases = arr(4).toInt,
        nDeaths = arr(5).toInt,
        country = arr(6),
        continent = arr(10)
      )
    })

In [ ]:
def infectionRDD = infections(spark.sparkContext.textFile("../datasets/data.csv"))

In [ ]:
org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this)
case class Population(
    country : String, 
    population : Int, 
    density : Int, 
    land_area: Int, 
    ) 
extends Serializable

In [ ]:
def populationData = spark.sparkContext.textFile("../datasets/population_by_country_2020.csv")

In [ ]:
def population(lines : RDD[String]) : RDD[Population] =
    lines.mapPartitionsWithIndex(
                   (index, it) => if (index == 0) it.drop(1) else it,
                    preservesPartitioning = true
                 )
    .map(line => {
      val arr = line.split(",")
      Population(
        country = arr(0),
        population = arr(1).toInt,
        density = arr(4).toInt,
        land_area = arr(5).toInt,
      )
    })

In [ ]:
def populationRDD = population(populationData)

### Un join computacionalmente pesado desde el principio ya que cruza todos los datos sin quedarnos con los que nos interesen

Spark no me deja hacer un Join de RDD que no sean pair RDD así que tenemos que construirlo

In [ ]:
// populationRDD.join(infectionRDD)

Construyo Pair RDDs conservando todos los datos

In [ ]:
def populationByCountry = populationRDD.map(
    x => (x.country,x))

def infectionByCountry = 
      infectionRDD.map(x => (x.country,x))

Hago el Join y agrupo por paises

In [ ]:
def joinedRDD = infectionByCountry.join(populationByCountry).groupByKey()

Finalmente calculo la media

In [ ]:
joinedRDD.mapValues(
    x => x.map( 
        line => line._1.nCases.toFloat / line._2.land_area.toFloat
    )).mapValues(
    x => x.sum / x.size
)

Lo hago todo en una única operación para calcular el tiempo de ejecución

In [ ]:
val notOptimizedRDD =
    infectionByCountry.join(populationByCountry)
    .groupByKey()
    .mapValues(
    x => x.map( 
        line => line._1.nCases.toFloat / line._2.land_area.toFloat)
    ).mapValues(
        x => x.sum / x.size
    )

#### Para optimizar un poco esta consulta:

Despejo solo los datos que me interesan para trabajar con Pair RDDs y optimizar la consulta

In [ ]:
def countriesAndLandArea = populationRDD.map(
    x => (x.country,x.land_area))

In [ ]:
def countriesAndCases = 
      infectionRDD.map(x => (x.country,x.nCases))
      .groupByKey()

Ejecuto un join y trabajo para calcular primero la media de infecciones por Km2 diaria, 
para luego calcular la media total

In [ ]:
def average = countriesAndCases.join(countriesAndLandArea)

In [ ]:
average.mapValues(
    x => x._1.map(
        y => (y.toFloat / x._2.toFloat)
    )).mapValues(
    x => x.sum/x.size
)

Lo hago todo en una única operación para calcular el tiempo de ejecución

In [ ]:
def meanInfectionsRDD =
countriesAndCases.join(countriesAndLandArea)   
.mapValues(
    x => x._1.map(
        y => (y.toDouble / x._2.toDouble)
    )).mapValues(
    x => x.sum / x.size
)

## Consulta con DataSet

In [ ]:
def infectionDS = spark.read
.option("header", "true")
.option("charset", "UTF8")
.option("delimiter",",")
.option("inferSchema", "true")
.csv("../datasets/covidworldwide.csv")
.withColumnRenamed("countriesAndTerritories","Country")
.as[(String,String,String,String,Double,Double,String,String,String,String,String,String)]

In [19]:
def populationDS = spark.read
.option("header", "true")
.option("charset", "UTF8")
.option("delimiter",",")
.option("inferSchema", "true")
.csv("../datasets/population_by_country_2020.csv")
.withColumnRenamed("Country (or dependency)","Country")
.withColumnRenamed("Population (2020)","Population")
.as[(String,Float,String,Float,Float,Float,Double,String,String,String,String)]

defined function populationDS

In [20]:
def meanInfectionsperKM2DS = 
infectionDS.join(populationDS, "Country")
        .select($"Country",
                $"dateRep" as "date",
                $"cases",
                $"Land Area (Km\u00b2)",
                $"cases" / $"Land Area (Km\u00b2)" as "infection Per Km\u00b2")
        .groupBy("Country")
        .agg(round(avg("infection Per Km\u00b2"),10).as[Float])
        .orderBy(desc("round(avg(infection Per Km²), 10)"))
        .as[(String,Double)]

defined function meanInfectionsperKM2DS

## Consulta con DataFrame

In [21]:
def dfCovid = spark.read
.option("header", "true")
.option("charset", "UTF8")
.option("delimiter",",")
.option("inferSchema", "true")
.csv("../datasets/covidworldwide.csv")

defined function dfCovid

In [22]:
def dfPopulation = spark.read
.option("header", "true")
.option("charset", "UTF8")
.option("delimiter",",")
.option("inferSchema", "true")
.csv("../datasets/population_by_country_2020.csv")
.withColumnRenamed("Country (or dependency)","Country")
.withColumnRenamed("Population (2020)","Population")

defined function dfPopulation

Modifico los datos de entrada para que el formato fecha se adecue al TimeStamp de Spark

In [23]:
def dfCovidClean = dfCovid
    .select($"*",$"dateRep",translate($"dateRep","/","-").as("date"))
    .drop("dateRep")

defined function dfCovidClean

In [24]:
def dfCovidDate = dfCovidClean
    .select($"*",col("date"),to_date(col("date"),"dd-MM-yyyy").as("to_date"))

defined function dfCovidDate

Hago una consulta de prueba para obtener la media solo de los casos en España

In [25]:
def spainCovid = dfCovid
    .select("dateRep","cases")
    .where("countriesAndTerritories == 'Spain'").toDF.agg(avg("cases"))

defined function spainCovid

### Finalmente ejecuto la consulta de nuestro caso de uso, infecciones por Km2

In [26]:
def meanInfectionsperKM2DF = 
dfCovid.join(dfPopulation, $"country" === $"countriesAndTerritories")
        .select($"country",
                $"dateRep" as "date",
                $"cases",
                $"Land Area (Km\u00b2)",
                $"cases" / $"Land Area (Km\u00b2)" as "infection Per Km\u00b2")
        .groupBy("country")
        .avg("infection Per Km\u00b2")
        .orderBy(desc("avg(infection Per Km²)"))

defined function meanInfectionsperKM2DF

# Visualización de eficiencia

In [29]:
val (x, y) = Seq(
    "Not Optimized RDD" -> runWithOutput(notOptimizedRDD.collect()),
    "RDD" -> runWithOutput(meanInfectionsRDD.collect()),
    "DataSet" -> runWithOutput(meanInfectionsperKM2DS.collect),
    "DataFrame" -> runWithOutput(meanInfectionsperKM2DF.collect)
).unzip

Bar(x, y).plot()

collect at cmd28.sc:2

2 / 2

Took 1493


map at cmd13.sc:2

2 / 2

...

200 / 200

collect at cmd28.sc:5

157 / 157

Took 2468


x: Seq[String] = List("Not Optimized RDD", "RDD", "DataSet", "DataFrame")
y: Seq[Int] = List(1493, 637, 2574, 2468)
res28_1: String = "plot-d81ee546-c799-481c-8bb1-82f4fd1d5cfd"

In [ ]:
spark.time(notOptimizedRDD.collect())

In [ ]:
spark.time(meanInfectionsRDD.collect())

In [ ]:
spark.time(meanInfectionsperKM2DS.collect())

In [ ]:
spark.time(meanInfectionsperKM2DF.collect())

In [ ]:
ch.cern.sparkmeasure.StageMetrics(spark).runAndMeasure(notOptimizedRDD.collect)

In [ ]:
ch.cern.sparkmeasure.StageMetrics(spark).runAndMeasure(meanInfectionRDD.collect)

In [ ]:
ch.cern.sparkmeasure.StageMetrics(spark).runAndMeasure(meanInfectionsperKM2DS.collect)

In [ ]:
ch.cern.sparkmeasure.StageMetrics(spark).runAndMeasure(
    meanInfectionsperKM2DF.collect
    )

# Visualización de datos con plotly

In [30]:
val (x,y) = meanInfectionsperKM2DF.collect.map(r=>(r(0).toString, r(1).toString.toFloat)).toList.unzip
Bar(x, y).plot()

csv at cmd20.sc:6

1 / 1

csv at cmd20.sc:6

2 / 2

csv at cmd21.sc:6

...

collect at cmd29.sc:1

200 / 200

collect at cmd29.sc:1

157 / 157

x: List[String] = List(
  "Monaco",
  "Gibraltar",
  "Bahrain",
  "Singapore",
  "Maldives",
  "Malta",
  "Aruba",
  "Belgium",
  "Andorra",
  "Netherlands",
  "Guam",
  "Israel",
  "Luxembourg",
  "Lebanon",
  "Qatar",
  "Liechtenstein",
  "Bermuda",
  "Cura\u00e7ao",
  "Switzerland",
  "Kuwait",
  "Italy",
  "Slovenia",
  "Armenia",
  "Moldova",
  "Bangladesh",
  "Portugal",
  "Poland",
  "France",
  "Montenegro",
  "Austria",
  "Hungary",
  "Germany",
  "Serbia",
  "Jordan",
  "Spain",
  "Slovakia",
  "India",
  "Panama",
...
y: List[Float] = List(
  1.9763314F,
  0.39814815F,
  0.3360843F,
  0.23804082F,
  0.15914285F,
  0.12258171F,
  0.10505618F,
  0.05736941F,
  0.056567993F,
  0.051918827F,
  0.048558153F,
  0.047656838F,
  0.04645812F,
  0.04139474F,
  0.035090588F,
  0.03391577F,
  0.031925924F,
  0.03068482F,
  0.026920669F,
  0.023646325F,
  0.017908985F,
  0.016838819F,
  0.015314984F,
  0.013754947F,
  0.013703283F,
  0.013267124F,
  0.013058352F,
  0.012402372F,
  0.0113